![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/PySpark/1.PySpark_RDDs.ipynb)


# **1. PySpark RDDs**


# Introduction, Features and Operations of RDD

Overview

At a high level, every Spark application consists of a driver program that runs the user’s main function and executes various parallel operations on a cluster. The main abstraction Spark provides is a resilient distributed dataset (RDD), which is a collection of elements partitioned across the nodes of the cluster that can be operated on in parallel. RDDs are created by starting with a file in the Hadoop file system (or any other Hadoop-supported file system), or an existing Scala collection in the driver program, and transforming it. Users may also ask Spark to persist an RDD in memory, allowing it to be reused efficiently across parallel operations. Finally, RDDs automatically recover from node failures.

A second abstraction in Spark is shared variables that can be used in parallel operations. By default, when Spark runs a function in parallel as a set of tasks on different nodes, it ships a copy of each variable used in the function to each task. Sometimes, a variable needs to be shared across tasks, or between tasks and the driver program. Spark supports two types of shared variables: broadcast variables, which can be used to cache a value in memory on all nodes, and accumulators, which are variables that are only “added” to, such as counters and sums.

This guide shows each of these features in each of Spark’s supported languages. It is easiest to follow along with if you launch Spark’s interactive shell – either bin/spark-shell for the Scala shell or bin/pyspark for the Python one.
[source](https://spark.apache.org/docs/latest/rdd-programming-guide.html)


### Install PySpark


In [ ]:
# install PySpark
%pip install pyspark==3.5.6

Note: you may need to restart the kernel to use updated packages.


In [6]:
import pyspark

### Initializing Spark


In PySpark, the entry point to programming Spark with the Dataset and DataFrame API is the `SparkSession` class.

You might also see examples that use `SparkContext`, which was the entry point to Spark functionality in Spark 1.x.

To compare, here's a table that highlights the differences between `SparkSession` and `SparkContext`:

| Feature           | SparkSession                       | SparkContext                |
| ----------------- | ---------------------------------- | --------------------------- |
| Introduced in     | Spark 2.0                          | Spark 1.x                   |
| API Level         | High-level (DataFrames, SQL)       | Low-level (RDDs)            |
| Supports SQL      | Yes                                | No                          |
| Underlying engine | Wraps and uses SparkContext        | Core engine                 |
| Used for          | DataFrame, Dataset, SQL, Streaming | RDDs and cluster operations |

#### TL;DR

In conclusion, if you're working with DataFrames, Datasets, or SQL, you should use `SparkSession`.

If you're working with only RDDs (e.g., `sc.parallelize` and `rdd.map`, `rdd.filter`, etc.), you might still use `SparkContext`, but it's generally recommended to use `SparkSession` for new applications.


In [7]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("PySparkTutorial").getOrCreate()
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=PySparkTutorial>

In [ ]:
# ==>> DO NOT FORGET WHEN YOU'RE DONE>> spark.stop()

In [8]:
print("PySpark version: ", spark.version)
# 3.2.0

# Python Version: To retrieve Python version of SparkContext
print("Python version:  ", spark.sparkContext.pythonVer)

# Master: URL of the cluster or “local” string to run in local mode of SparkContext
print(spark.sparkContext.master)

PySpark version:  3.5.1
Python version:   3.12
local[*]


### **[Exercise]** Compare SparkSession and SparkContext

Run the code below to create a `SparkContext` instance and print its version, Python version, and master URL. Observe that `SparkSession` now encapsulates `SparkContext`, so you can access it via `spark.sparkContext` if you have a `SparkSession` instance.

#### Code

```python
from pyspark import SparkContext

sc = SparkContext(appName="PySparkTutorial", master= "local[*]")
sc = SparkContext.getOrCreate()
sc

# PySpark version: To retrieve PySpark version of SparkContext
print("PySpark version: ", sc.version)

# Python Version: To retrieve Python version of SparkContext
print("Python version:  ", sc.pythonVer)

# Master: URL of the cluster or “local” string to run in local mode of SparkContext
print(sc.master)
```


# Spark RDD (Resillient Distributed Datasets)

Spark revolves around the concept of a resilient distributed dataset (RDD), which is a fault-tolerant collection of elements that can be operated on in parallel. There are two ways to create RDDs: parallelizing an existing collection in your driver program, or referencing a dataset in an external storage system, such as a shared filesystem, HDFS, HBase, or any data source offering a Hadoop InputFormat.


### Creating RDDs

There are two ways to create RDDs,

- parallelizing an existing collection of objects in your driver program,

- External datasets (referencing a dataset in an external storage system, such as a shared file system, HDFS, HBase, or any data source offering a Hadoop Input Format.)


#### Parallelized Collections

Parallelized collections are created by calling SparkContext’s parallelize method on an existing iterable or collection in your driver program. The elements of the collection are copied to form a distributed dataset that can be operated on in parallel. For example, here is how to create a parallelized collection holding the numbers 1 to 5:


In [9]:
# parallelize() for creating RDDs from python lists

data = [1, 2, 3, 4, 5]
distData = sc.parallelize(data)
print(type(distData))

# collect() to retrieve all elements of the RDD
# Do not use collect() on large RDDs as it may cause out of memory error on the driver
distData.collect()

<class 'pyspark.rdd.RDD'>


[1, 2, 3, 4, 5]

#### External Datasets

Spark can create distributed datasets from any storage source supported by Hadoop, including your local file system, HDFS, Cassandra, HBase, Amazon S3, etc. Spark supports text files, SequenceFiles, and any other Hadoop InputFormat.

Text file RDDs can be created using SparkContext’s textFile method. This method takes a URI for the file (either a local path on the machine, or a hdfs://, s3a://, etc URI) and reads it as a collection of lines. Here is an example invocation:


In [10]:
# Run these lines to fetch the sample dataset if you are on Colab
!mkdir -p ./data
!wget -q -P ./data https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/PySpark/data/airport-codes.csv
!wget -q -P ./data https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/PySpark/data/news_category_test.csv

In [11]:
# textFile() for creating RDDs from existing file

rdd = sc.textFile("./data/news_category_test.csv")

In [12]:
rdd.collect()[:10]

['category,description',
 "Business,Unions representing workers at Turner   Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul.",
 'Sci/Tech," TORONTO, Canada    A second team of rocketeers competing for the  #36;10 million Ansari X Prize, a contest for privately funded suborbital space flight, has officially announced the first launch date for its manned rocket."',
 'Sci/Tech," A company founded by a chemistry researcher at the University of Louisville won a grant to develop a method of producing better peptides, which are short chains of amino acids, the building blocks of proteins."',
 'Sci/Tech," It\'s barely dawn when Mike Fitzpatrick starts his shift with a blur of colorful maps, figures and endless charts, but already he knows what the day will bring. Lightning will strike in places he expects. Winds will pick up, moist places will dry and flames will roar."',
 'Sci/Tech," Southern California\'s smog fighting agency went after emissions of the

## RDDs Operations


RDDs support two types of operations: transformations, which create a new dataset from an existing one, and actions, which return a value to the driver program after running a computation on the dataset. For example, map is a transformation that passes each dataset element through a function and returns a new RDD representing the results. On the other hand, reduce is an action that aggregates all the elements of the RDD using some function and returns the final result to the driver program (although there is also a parallel reduceByKey that returns a distributed dataset).

By default, each transformed RDD may be recomputed each time you run an action on it. However, you may also persist an RDD in memory using the persist (or cache) method, in which case Spark will keep the elements around on the cluster for much faster access the next time you query it. There is also support for persisting RDDs on disk, or replicated across multiple nodes.

#### Common RDD Transformations

**All transformations in Spark are lazy**, The transformations are only computed when an action requires a result to be returned to the driver program.

| **Transformations (lazy)** | **Documentation**                                                                                                        |
| -------------------------- | ------------------------------------------------------------------------------------------------------------------------ |
| `map`                      | [RDD.map() – PySpark](https://spark.apache.org/docs/3.5.6/api/python/reference/api/pyspark.RDD.map.html)                 |
| `filter`                   | [RDD.filter() – PySpark](https://spark.apache.org/docs/3.5.6/api/python/reference/api/pyspark.RDD.filter.html)           |
| `flatMap`                  | [RDD.flatMap() – PySpark](https://spark.apache.org/docs/3.5.6/api/python/reference/api/pyspark.RDD.flatMap.html)         |
| `reduceByKey`              | [RDD.reduceByKey() – PySpark](https://spark.apache.org/docs/3.5.6/api/python/reference/api/pyspark.RDD.reduceByKey.html) |
| `join`                     | [RDD.join() – PySpark](https://spark.apache.org/docs/3.5.6/api/python/reference/api/pyspark.RDD.join.html)               |
| `cogroup`                  | [RDD.cogroup() – PySpark](https://spark.apache.org/docs/3.5.6/api/python/reference/api/pyspark.RDD.cogroup.html)         |
| `repartition`              | [RDD.repartition() – PySpark](https://spark.apache.org/docs/3.5.6/api/python/reference/api/pyspark.RDD.repartition.html) |

#### Common RDD Actions

**Actions are eager operations that immediately trigger the execution of the transformations required** to compute the RDD. Actions return values to the driver program or write data to an external storage system.

| **Actions (eager)** | **Documentation**                                                                                                                  |
| ------------------- | ---------------------------------------------------------------------------------------------------------------------------------- |
| `count`             | [RDD.count() – PySpark](https://spark.apache.org/docs/3.5.6/api/python/reference/api/pyspark.RDD.count.html)                       |
| `reduce`            | [RDD.reduce() – PySpark](https://spark.apache.org/docs/3.5.6/api/python/reference/api/pyspark.RDD.reduce.html)                     |
| `collect`           | [RDD.collect() – PySpark](https://spark.apache.org/docs/3.5.6/api/python/reference/api/pyspark.RDD.collect.html)                   |
| `take`              | [RDD.take() – PySpark](https://spark.apache.org/docs/3.5.6/api/python/reference/api/pyspark.RDD.take.html)                         |
| `saveAsTextFile`    | [RDD.saveAsTextFile() – PySpark](https://spark.apache.org/docs/3.5.6/api/python/reference/api/pyspark.RDD.saveAsTextFile.html)     |
| `saveAsHadoopFile`  | [RDD.saveAsHadoopFile() – PySpark](https://spark.apache.org/docs/3.5.6/api/python/reference/api/pyspark.RDD.saveAsHadoopFile.html) |
| `countByValue`      | [RDD.countByValue() – PySpark](https://spark.apache.org/docs/3.5.6/api/python/reference/api/pyspark.RDD.countByValue.html)         |


### Examples


- collect() : Return all the elements of the dataset as an array at the driver program. This is usually useful after a filter or other operation that returns a sufficiently small subset of the data.


In [13]:
rdd.collect()[:5]

['category,description',
 "Business,Unions representing workers at Turner   Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul.",
 'Sci/Tech," TORONTO, Canada    A second team of rocketeers competing for the  #36;10 million Ansari X Prize, a contest for privately funded suborbital space flight, has officially announced the first launch date for its manned rocket."',
 'Sci/Tech," A company founded by a chemistry researcher at the University of Louisville won a grant to develop a method of producing better peptides, which are short chains of amino acids, the building blocks of proteins."',
 'Sci/Tech," It\'s barely dawn when Mike Fitzpatrick starts his shift with a blur of colorful maps, figures and endless charts, but already he knows what the day will bring. Lightning will strike in places he expects. Winds will pick up, moist places will dry and flames will roar."']

In [14]:
%%time
%%capture
rdd.collect()

CPU times: user 36.4 ms, sys: 3 ms, total: 39.4 ms
Wall time: 197 ms


In [15]:
numbRDD = sc.parallelize([0, 1, 2, 3, 4])

# Create map() transformation to cube numbers
cubedRDD = numbRDD.map(lambda x: x**3)

# Collect the results
numbers_all = cubedRDD.collect()

# Print the numbers from numbers_all
for numb in numbers_all:
    print(numb)

0
1
8
27
64


- filter() : Return a new dataset formed by selecting those elements of the source on which func returns true.


In [16]:
rdd.filter(lambda x: "Business" in x).collect()[:10]

["Business,Unions representing workers at Turner   Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul.",
 'Business," Apparel retailers are hoping their back to school fashions will make the grade among style conscious teens and young adults this fall, but it could be a tough sell, with students and parents keeping a tighter hold on their wallets."',
 'Business," The dollar dipped to a four week low  against the euro on Monday before rising slightly on  profit taking, but steep oil prices and weak U.S. data  continued to fan worries about the health of the world\'s  largest economy."',
 'Business," U.S. Treasury debt prices slipped on  Monday, though traders characterized the move as profit taking  rather than any fundamental change in sentiment."',
 'Business, The dollar extended gains against the  euro on Monday after a report on flows into U.S. assets showed  enough of a rise in foreign investments to offset the current  account gap for the month.

- count() Return the number of elements in the dataset.


In [17]:
rdd.count()

7601

- map() Return a new distributed dataset formed by passing each element of the source through a function func


In [18]:
LineLength = rdd.map(lambda x: len(x))
print(LineLength.count())
print(LineLength.collect()[:5])

7601
[20, 136, 234, 221, 279]


- take(n) Return an array with the first n elements of the dataset.


In [19]:
rdd.take(5)

['category,description',
 "Business,Unions representing workers at Turner   Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul.",
 'Sci/Tech," TORONTO, Canada    A second team of rocketeers competing for the  #36;10 million Ansari X Prize, a contest for privately funded suborbital space flight, has officially announced the first launch date for its manned rocket."',
 'Sci/Tech," A company founded by a chemistry researcher at the University of Louisville won a grant to develop a method of producing better peptides, which are short chains of amino acids, the building blocks of proteins."',
 'Sci/Tech," It\'s barely dawn when Mike Fitzpatrick starts his shift with a blur of colorful maps, figures and endless charts, but already he knows what the day will bring. Lightning will strike in places he expects. Winds will pick up, moist places will dry and flames will roar."']

## Partition, Repartition and Coalesce


- saveAsTextFile(path) Write the elements of the dataset as a text file (or set of text files) in a given directory in the local filesystem, HDFS or any other Hadoop-supported file system. Spark will call toString on each element to convert it to a line of text in the file.


In [20]:
# delete existing savedData folder
#! rm -R savedData

In [21]:
# Understanding Partitioning in PySpark
# Save RDD Data to HDFS
# textFile() method

rdd3 = sc.textFile("./data/airport-codes.csv", minPartitions=5)

rdd3.saveAsTextFile("./savedData/")
#  hadoop fs -cat /savedData/part-00000
#  hadoop fs -cat /savedData/part-00001
#  hadoop fs -cat /savedData/part-00002
#  hadoop fs -cat /savedData/part-00003
#  hadoop fs -cat /savedData/part-00004

print(rdd3.getNumPartitions())

5


In [22]:
# parallelize() method

numRDD = sc.parallelize(range(10), numSlices=3)

print(numRDD.getNumPartitions())

3


In [23]:
print(numRDD.collect())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


- glom() return an RDD created by coalescing all elements within each partition into a list. https://medium.com/parrot-prediction/partitioning-in-apache-spark-8134ad840b0


In [24]:
# Default Partition Number is 2

rdd = sc.parallelize(range(10))

print("Number of partitions: {}".format(rdd.getNumPartitions()))
print("Partitions structure: {}".format(rdd.glom().collect()))

Number of partitions: 2
Partitions structure: [[0, 1, 2, 3, 4], [5, 6, 7, 8, 9]]


In [25]:
rdd = sc.parallelize(range(10), numSlices=4)

print("Number of partitions: {}".format(rdd.getNumPartitions()))
print("Partitions structure: {}".format(rdd.glom().collect()))

Number of partitions: 4
Partitions structure: [[0, 1], [2, 3, 4], [5, 6], [7, 8, 9]]


In [26]:
rdd = sc.parallelize(range(10), 4)

print("Number of partitions: {}".format(rdd.getNumPartitions()))
print("Partitions structure: {}".format(rdd.glom().collect()))

Number of partitions: 4
Partitions structure: [[0, 1], [2, 3, 4], [5, 6], [7, 8, 9]]


- if you are increasing the number of partitions use **repartition()** (performing full shuffle),


In [27]:
repartRdd = rdd.repartition(5)

print("Number of partitions: {}".format(repartRdd.getNumPartitions()))
print("Partitions structure: {}".format(repartRdd.glom().collect()))

Number of partitions: 5
Partitions structure: [[], [], [], [7, 8, 9], [0, 1, 2, 3, 4, 5, 6]]


- If you are decreasing the number of partitions in this RDD, consider
  using **coalesce()**, which can avoid performing a shuffle.


In [28]:
repartRdd = rdd.coalesce(2)

print("Number of partitions: {}".format(repartRdd.getNumPartitions()))
print("Partitions structure: {}".format(repartRdd.glom().collect()))

Number of partitions: 2
Partitions structure: [[0, 1, 2, 3, 4], [5, 6, 7, 8, 9]]


## Passing Functions to Spark

Spark’s API relies heavily on passing functions in the driver program to run on the cluster. There are three recommended ways to do this:

Lambda expressions, for simple functions that can be written as an expression. (Lambdas do not support multi-statement functions or statements that do not return a value.)
Local defs inside the function calling into Spark, for longer code.
Top-level functions in a module.
For example, to pass a longer function than can be supported using a lambda, consider the code below:


In [29]:
"""MyScript.py"""

if __name__ == "__main__":

    def myFunc(s):

        return len(s)


rdd3.map(myFunc)

PythonRDD[32] at RDD at PythonRDD.scala:53

In [30]:
print(rdd3.map(myFunc).count())

print(rdd3.map(myFunc).collect()[:5])

55114
[116, 100, 98, 101, 106]


In [31]:
# Read a CSV File
# Writing a Python Function to Parse CSV Lines
import csv
from io import StringIO


def parseCSV(csvRow):
    data = StringIO(csvRow)
    dataReader = csv.reader(data, lineterminator="")
    return next(dataReader)


csvRow = "p,s,r,p"
parseCSV(csvRow)

['p', 's', 'r', 'p']

In [32]:
# Read csv file and Creating a Paired RDD
filamentRDD = sc.textFile("./data/airport-codes.csv", 4)
filamentRDDCSV = filamentRDD.map(parseCSV)
filamentRDDCSV.take(1)

[['ident',
  'type',
  'name',
  'elevation_ft',
  'continent',
  'iso_country',
  'iso_region',
  'municipality',
  'gps_code',
  'iata_code',
  'local_code',
  'coordinates']]

## Anonymous functions

Lambda functions are anonymous functions in Python


In [33]:
## Transformations (lazy evaluation)

# map() transformation applies a function to all elements in the RDD

import math

numRDD = sc.parallelize(range(100), numSlices=3)

RDD = sc.parallelize([0, 1, 1.6, 2, 3, 3.14])
RDD_map = RDD.map(lambda x: math.sin(x))

RDD_map.collect()

[0.0,
 0.8414709848078965,
 0.9995736030415051,
 0.9092974268256817,
 0.1411200080598672,
 0.0015926529164868282]

In [34]:
# filter() transformation returns a new RDD with only the elements that pass the condition

RDD = sc.parallelize([0, 1, 2, 3, 4])

RDD_filter = RDD.filter(lambda x: x >= 2)

RDD_filter.collect()

[2, 3, 4]

In [35]:
# flatMap() transformation returns multiple values for each element in the original RDD

"""
Why are we using flatMap, rather than map?

The reason is that the operation line.split(" ") generates a list of strings,
so had we used map the result would be an RDD of lists of words. Not an RDD of words.

The difference between map and flatMap is that the second expects to get a list as the result
from the map and it concatenates the lists to form the RDD.
"""

RDD = sc.parallelize(["hello world", "how are you"])

RDD_flatmap = RDD.flatMap(lambda x: x.split(" "))

RDD_flatmap.collect()

['hello', 'world', 'how', 'are', 'you']

## Introduction to pair RDDs in PySpark

Two common ways to create pair RDDs

    From a list of key-value tuple
    From a regular RDD

Get the data into key/value form for paired RDD


In [36]:
my_tuple = [("Sam", 23), ("Mary", 34), ("Peter", 25)]

pairRDD_tuple = sc.parallelize(my_tuple)

pairRDD_tuple.collect()

[('Sam', 23), ('Mary', 34), ('Peter', 25)]

In [37]:
my_list = ["Sam 23", "Mary 34", "Peter 25"]

regularRDD = sc.parallelize(my_list)

pairRDD_RDD = regularRDD.map(lambda s: (s.split(" ")[0], s.split(" ")[1]))

pairRDD_RDD.collect()

[('Sam', '23'), ('Mary', '34'), ('Peter', '25')]

In [38]:
#  Fetching Values from a Paired RDD
pairRDD_RDD_Values = pairRDD_RDD.values()
pairRDD_RDD_Values.collect()

['23', '34', '25']

In [39]:
#  Fetching Keys from a Paired RDD
pairRDD_RDD_Keys = pairRDD_RDD.keys()
pairRDD_RDD_Keys.collect()

['Sam', 'Mary', 'Peter']

## Join


In [40]:
people_rdd = sc.parallelize(
    [
        "name, age, occupation",
        "Alex, 25, student",
        "Mary, 30, scientist",
        "Brown, 45, teacher",
    ]
)

transactions_rdd = sc.parallelize(
    [
        "product, price, name",
        "Book, 10, Brown",
        "Schoolbag, 12, Alex",
        "Pen, 4, Brown",
        "Mouse, 8, Mary",
    ]
)


# Expand each row from one long string into a list of discrete entries
people_rdd = people_rdd.map(lambda line: line.split(", "))
transactions_rdd = transactions_rdd.map(lambda line: line.split(", "))

people_rdd.collect()

[['name', 'age', 'occupation'],
 ['Alex', '25', 'student'],
 ['Mary', '30', 'scientist'],
 ['Brown', '45', 'teacher']]

In [41]:
# Remove header row, since RDD operations are low-level Spark operations that do not rely on schema
people_header = people_rdd.first()
transactions_header = transactions_rdd.first()

people_rdd = people_rdd.filter(lambda line: line != people_header)
transactions_rdd = transactions_rdd.filter(lambda line: line != transactions_header)

people_rdd.collect()
transactions_rdd.collect()

[['Book', '10', 'Brown'],
 ['Schoolbag', '12', 'Alex'],
 ['Pen', '4', 'Brown'],
 ['Mouse', '8', 'Mary']]

In [42]:
# Format each RDD as (K, V) to prepare for the join operation

people_rdd = people_rdd.map(lambda line: (line[0], line[1] + ", " + line[2]))
transactions_rdd = transactions_rdd.map(
    lambda line: (line[2], line[0] + ", " + line[1])
)

people_rdd.collect()

[('Alex', '25, student'), ('Mary', '30, scientist'), ('Brown', '45, teacher')]

In [43]:
join = people_rdd.join(transactions_rdd)
join.collect()

[('Mary', ('30, scientist', 'Mouse, 8')),
 ('Alex', ('25, student', 'Schoolbag, 12')),
 ('Brown', ('45, teacher', 'Book, 10')),
 ('Brown', ('45, teacher', 'Pen, 4'))]

## Transformations on pair RDDs

All regular transformations work on pair RDD

Have to pass functions that operate on tuples rather than on individual elements

Examples of paired RDD Transformations


In [44]:
# we can use user functions to map on RDD


def get_Squares(num):
    return num**2


numbRDD = sc.parallelize([0, 1, 2, 3, 4, 3, 2, 1, 0])

numbRDD.map(get_Squares).collect()

[0, 1, 4, 9, 16, 9, 4, 1, 0]

In [45]:
# distinct() to find the distinct numbers

numbRDD.distinct().collect()

[0, 2, 4, 1, 3]

In [46]:
#  intersection()

numbRDD2 = sc.parallelize([1, 2, 3, 5])

numbRDD.intersection(numbRDD2).collect()

[1, 2, 3]

In [47]:
# calculating basic stats

numbRDD = sc.parallelize([1, 2, 3, 4, 2, 5, 1])

print(numRDD.min())

print(numRDD.max())

print(numRDD.sum())

print(numRDD.mean())

print(numRDD.variance())

print(numRDD.stdev())

print(numRDD.stats())

print(numRDD.stats().asDict())

0
99
4950
49.5
833.25
28.86607004772212
(count: 100, mean: 49.5, stdev: 28.86607004772212, max: 99.0, min: 0.0)
{'count': 100, 'mean': 49.5, 'sum': 4950.0, 'min': np.float64(0.0), 'max': np.float64(99.0), 'stdev': np.float64(29.011491975882016), 'variance': 841.6666666666666}


In [48]:
# reduceByKey() transformation combines values with the same key

# It runs parallel operations for each key in the dataset

# It is a transformation and not action

regularRDD = sc.parallelize(
    [
        ("Messi", 23),
        ("Ronaldo", 34),
        ("Neymar", 22),
        ("Messi", 24),
        ("Ronaldo", 24),
        ("Neymar", 24),
    ]
)

pairRDD_reducebykey = regularRDD.reduceByKey(lambda x, y: x + y)

pairRDD_reducebykey.collect()

[('Messi', 47), ('Ronaldo', 58), ('Neymar', 46)]

In [49]:
# groupbykey() groups all the values with the same key in the pair RDD

airports = [("US", "JFK"), ("UK", "LHR"), ("FR", "CDG"), ("US", "SFO")]

regularRDD = sc.parallelize(airports)

pairRDD_group = regularRDD.groupByKey().collect()

for cont, air in pairRDD_group:
    print(cont, list(air))

UK ['LHR']
US ['JFK', 'SFO']
FR ['CDG']


In [50]:
# join() transformation joins the two pair RDDs based on their key

x = sc.parallelize([("a", 1), ("b", 4), ("c", 2), ("d", 6)])

y = sc.parallelize([("a", 2), ("a", 3), ("c", 3), ("d", 5)])

sorted(x.join(y).collect())

[('a', (1, 2)), ('a', (1, 3)), ('c', (2, 3)), ('d', (6, 5))]

In [51]:
# reduce(func) action is used for aggregating the elements of a regular RDD

# The function should be commutative and associative

# An example of reduce() action in PySpark

from operator import add

print(sc.parallelize([1, 2, 3, 4, 5]).reduce(add))

15


In [52]:
sc.parallelize((2 for _ in range(10))).map(lambda x: 1).cache().reduce(add)

10

In [53]:
# countByKey() only available for type (K, V)

# countByKey() action counts the number of elements for each key

# Example of countByKey() on a simple list

rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 2), ("b", 3), ("a", 1)])

for key, val in rdd.countByKey().items():
    print(key, val)

a 3
b 2


In [54]:
# collectAsMap() return the key-value pairs in the RDD as a dictionary

# Example of collectAsMap() on a simple tuple

sc.parallelize([(1, 2), (3, 4)]).collectAsMap()

{1: 2, 3: 4}

In [55]:
# word count example

text_file = sc.textFile("./data/airport-codes.csv")
counts_rdd = (
    text_file.flatMap(lambda line: line.split(" "))
    .map(lambda word: (word, 1))
    .reduceByKey(lambda a, b: a + b)
)

# print the word frequencies in descending order

counts_rdd.map(lambda x: (x[1], x[0])).sortByKey(ascending=False).collect()[:20]

[(1871, 'Hospital'),
 (1392, 'Air'),
 (1314, 'Municipal'),
 (1247, 'Ranch'),
 (1023, 'de'),
 (1009, 'Center'),
 (1003, 'Seaplane'),
 (1000, 'International'),
 (872, 'County'),
 (761, 'Medical'),
 (672, 'Regional'),
 (609, 'Lake'),
 (582, 'Farm'),
 (529, 'Memorial'),
 (497, 'Landing'),
 (451, 'De'),
 (385, 'do'),
 (373, 'Creek'),
 (360, 'Island'),
 (334, 'Do')]

In [56]:
# bigrams and word frequencies

sentences = (
    sc.textFile("./data/airport-codes.csv")
    .glom()
    .map(lambda x: " ".join(x))
    .flatMap(lambda x: x.split("."))
)

bigrams = sentences.map(lambda x: x.split()).flatMap(
    lambda x: [((x[i], x[i + 1]), 1) for i in range(0, len(x) - 1)]
)

freq_bigrams = (
    bigrams.reduceByKey(lambda x, y: x + y).map(lambda x: (x[1], x[0])).sortByKey(False)
)

freq_bigrams.take(10)

[(683, ('Medical', 'Center')),
 (282, ('Memorial', 'Hospital')),
 (257, ('di', 'Volo')),
 (192, ('Lake', 'Seaplane')),
 (130, ('Regional', 'Medical')),
 (124, ('Community', 'Hospital')),
 (110, ('Air', 'Force')),
 (101, ('General', 'Hospital')),
 (83, ('Building', 'Heliport,,AS,KR,KR-11,Seoul,,,,"37')),
 (68, ('County', 'Hospital'))]

In [57]:
# spark.stop()

# Resources

1. https://spark.apache.org/docs/latest/rdd-programming-guide.html
2. https://spark.apache.org/docs/latest/api/python/getting_started/quickstart_df.html#
3. https://github.com/vkocaman/PySpark_Essentials_March_2019
4. https://github.com/sundarramamurthy/pyspark
5. https://towardsdatascience.com/beginners-guide-to-pyspark-bbe3b553b79f
6. https://www.guru99.com/pyspark-tutorial.html
7. https://towardsdatascience.com/exploratory-data-analysis-eda-with-pyspark-on-databricks-e8d6529626b1
8. https://www.datacamp.com/community/tutorials/apache-spark-tutorial-machine-learning
